In [161]:
from __future__ import print_function
import ipywidgets as widgets
from traitlets import Unicode, validate, CBytes, Float, CInt
import math

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

    value = Float().tag(sync=True)
    duration = Float().tag(sync=True)
    interval = Float().tag(sync=True)

    msvalue = CInt().tag(sync=True)
    msduration = CInt().tag(sync=True)
    msinterval = CInt().tag(sync=True)

    midi = CBytes().tag(sync=True)
    
    def __init__(self, **kwargs):
        super(widgets.DOMWidget, self).__init__(**kwargs)
        #x = widgets.Button( icon='play')
        

In [162]:
%%javascript
require.config({
    paths: {
        webaudiofontplayer: '../files/webaudiofont/npm/dist/WebAudioFontPlayer',
        midifile: '../files/webaudiofont/examples/MIDIFile'
    }
});

<IPython.core.display.Javascript object>

In [210]:
%%javascript
require.undef('hello');

define('hello', ["@jupyter-widgets/base", 
                 'webaudiofontplayer', 
                 'midifile'], function(widgets) {

    var audioContext = null;
    var player = null;
    var reverberator = null;
    var songStart = 0;
    var input = null;
    var currentSongTime = 0;
    var song = null;
    var stepDuration = 44e-3;

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.msvalue_changed();
            this.model.on('change:msvalue', this.msvalue_changed, this);
            this.midi_changed();
            this.model.on('change:midi', this.midi_changed, this);
        },

        msvalue_changed: function() {
            var msvalue = this.model.get('msvalue');
            currentSongTime = msvalue / 1000;
            if (currentSongTime > 0.0) {
                if (songStart == 0.0)
                    songStart = audioContext.currentTime;
                this.tick();
            }
            else {
                songStart = 0.0;
            }
        },
        
        midi_changed: function() {
            var midi = this.model.get('midi').buffer;
            var midiFile = new MIDIFile( midi);
            songStart = 0.0;
            song = midiFile.parseSong();
            this.startLoad()
            this.updateModel()
        },

        updateModel: function() {
            this.model.set('duration', song.duration)
            this.model.set('msduration', Math.round( Math.ceil( song.duration / stepDuration) * stepDuration * 1000))
            this.model.set('interval', stepDuration)
            this.model.set('msinterval', Math.round( stepDuration * 1000))
            this.touch()
        },
        
        tick: function() {
            this.sendNotes(currentSongTime - stepDuration, currentSongTime);
        },
        
        startLoad: function() {
            console.log(song);
            var AudioContextFunc = window.AudioContext || window.webkitAudioContext;
            audioContext = new AudioContextFunc();
            player = new WebAudioFontPlayer();
            reverberator = player.createReverberator(audioContext);
            reverberator.output.connect(audioContext.destination);
            input = reverberator.input;
            for (var i = 0; i < song.tracks.length; i++) {
                var nn = player.loader.findInstrument(song.tracks[i].program);
                var info = player.loader.instrumentInfo(nn);
                song.tracks[i].info = info;
                song.tracks[i].id = nn;
                player.loader.startLoad(audioContext, info.url, info.variable);
            }
            for (var i = 0; i < song.beats.length; i++) {
                var nn = player.loader.findDrum(song.beats[i].n);
                var info = player.loader.drumInfo(nn);
                song.beats[i].info = info;
                song.beats[i].id = nn;
                player.loader.startLoad(audioContext, info.url, info.variable);
            }
            player.loader.waitLoad(function () {
            });
        },
        
        sendNotes: function(start, end) {
            for (var t = 0; t < song.tracks.length; t++) {
                var track = song.tracks[t];
                for (var i = 0; i < track.notes.length; i++) {
                    if (track.notes[i].when >= start && track.notes[i].when < end) {
                        var when = songStart + track.notes[i].when;
                        var duration = track.notes[i].duration;
                        if (duration > 3) {
                            duration = 3;
                        }
                        var instr = track.info.variable;
                        var v = track.volume / 7;
                        player.queueWaveTable(audioContext, input, window[instr], when, track.notes[i].pitch, duration, v, track.notes[i].slides);
                    }
                }
            }
            for (var b = 0; b < song.beats.length; b++) {
                var beat = song.beats[b];
                for (var i = 0; i < beat.notes.length; i++) {
                    if (beat.notes[i].when >= start && beat.notes[i].when < end) {
                        var when = songStart + beat.notes[i].when;
                        var duration = 1.5;
                        var instr = beat.info.variable;
                        var v = beat.volume / 2;
                        player.queueWaveTable(audioContext, input, window[instr], when, beat.n, duration, v);
                    }
                }
            }
        },
    });

    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [212]:
with open( './webaudiofont/examples/midi/123.mid', 'rb') as f:
    w = HelloWidget( midi = f.read())

play = widgets.Play()
widgets.jslink( (w, 'msinterval'), (play, 'interval'))
widgets.jslink( (w, 'msduration'), (play, 'max'))
widgets.jslink( (w, 'msinterval'), (play, 'step'))
widgets.jslink( (w, 'msvalue'), (play, 'value'))

slider = widgets.IntSlider()
widgets.jslink((play, 'step'), (slider, 'step'))
widgets.jslink((play, 'max'), (slider, 'max'))
widgets.jslink((play, 'value'), (slider, 'value'))

widgets.HBox([w, play, slider])

In [166]:
# https://ipython-books.github.io/34-creating-custom-jupyter-notebook-widgets-in-python-html-and-javascript/